# Script for transforming raw mobility data to edge table and trips per day and district

## Specify identifier for output files

In [3]:
identifier = 'teralytics_2019_20_'

## Import libraries

In [4]:
import pandas as pd
import numpy as np

## Read in raw data from file

Read in file into dataframe:

In [5]:
df_raw = pd.DataFrame()

for chunk in pd.read_csv('data/deldd-1-taeglich-aktualisierte-langdistanz-reisen-giz-counts.csv', chunksize=10**6):
    df_raw = df_raw.append(chunk)

df_raw.reset_index(inplace=True, drop=True)

In [6]:
df_raw.head(5)

,Bucket,StartId,StartName,EndId,EndName,Mode of Transport,Trip distance (km),Count
0,2020-09-20 (Sonntag),6241300,Havelland,6242600,Diepholz - Landkreis Diepholz,Road,264,10
1,2020-09-20 (Sonntag),6245100,Kaiserslautern - Landkreis Kaiserslautern,6254000,Emsland - Landkreis Emsland,Road,366,9
2,2020-09-20 (Sonntag),6236900,Potsdam,6267300,Osnabrück - Landkreis Osnabrück,Train,338,6
3,2020-09-20 (Sonntag),6249601,Wiesbaden - Mainz-Kostheim,6241701,Paderborn,Road,194,14
4,2020-09-20 (Sonntag),6238700,Merzig-Wadern - Landkreis Merzig-Wadern,6234700,Ludwigshafen am Rhein,Train,123,6


Drop columns we don't need:

In [7]:
df_prep = df_raw.drop(columns=['Mode of Transport','Trip distance (km)', 'StartName', 'EndName'])

Specify columns names with origin and destination districts, dates and mobility counts:

In [8]:
origin_col_name = 'StartId'
destination_col_name = 'EndId'
count_col_name = 'Count'


## OSM ID to AGS5

In [9]:
o2a_df = pd.read_csv('data/OSM_2_AGS_v2.csv')

In [10]:
osm_2_ags_dict = dict(zip(o2a_df.OSM_ID,o2a_df.AGS_5))

In [11]:
df_prep['orig_ags5'] = df_prep[origin_col_name].map(osm_2_ags_dict)
df_prep['dest_ags5'] = df_prep[destination_col_name].map(osm_2_ags_dict)

In [12]:
import locale
locale.setlocale(category=locale.LC_ALL,locale="German")

'German_Germany.1252'

In [13]:
df_prep['date'] = pd.to_datetime(df_prep['Bucket'], format = '%Y-%m-%d (%A)')

In [14]:
df_prep.drop(columns=[origin_col_name,destination_col_name, "Bucket"], inplace=True)

In [15]:
df_prep.head(5)

,Count,orig_ags5,dest_ags5,date
0,10,12063,3251,2020-09-20
1,9,7335,3454,2020-09-20
2,6,12054,3459,2020-09-20
3,14,6414,5774,2020-09-20
4,6,10042,7314,2020-09-20


## Create edge table

In [16]:
edge_table = df_prep.groupby(['date','orig_ags5','dest_ags5'])[count_col_name].sum().reset_index()

Save edge table to CSV-file using the specifier defined above:

In [17]:
file_name = identifier + 'edge_table.csv'
edge_table.to_csv(file_name, index=False)

In case there's a problem later on, reading in this CSV will serve as a checkpoint:

In [67]:
#edge_table = pd.read_csv(file_name)
#edge_table.tail(10)

,date,orig_ags5,dest_ags5,Count
29558204,2020-09-20,16077,16068,59
29558205,2020-09-20,16077,16069,27
29558206,2020-09-20,16077,16070,112
29558207,2020-09-20,16077,16071,88
29558208,2020-09-20,16077,16072,7
29558209,2020-09-20,16077,16073,137
29558210,2020-09-20,16077,16074,420
29558211,2020-09-20,16077,16075,171
29558212,2020-09-20,16077,16076,2459
29558213,2020-09-20,16077,16077,112818


## Create table for trips per district and day

Create Boolean Series to subset data into within- and between-district mobility:

In [18]:
is_internal = edge_table.orig_ags5 == edge_table.dest_ags5

Create multi-indexed Series for outgoing, incoming and internal mobility per district and day:

In [19]:
outgoing_mobility = edge_table[~is_internal].groupby(['date', 'orig_ags5'])[count_col_name].sum().reset_index()
incoming_mobility = edge_table[~is_internal].groupby(['date', 'dest_ags5'])[count_col_name].sum().reset_index()
internal_mobility = edge_table[ is_internal].groupby(['date', 'orig_ags5'])[count_col_name].sum().reset_index()

Merge all three into one dataframe, clean that dataframe up, and add total mobility:

In [20]:
external_mobility = pd.merge(outgoing_mobility, incoming_mobility, 
                   left_on = ['date', 'orig_ags5'], right_on = ['date', 'dest_ags5'],
                   suffixes = ('_outgoing','_incoming'))

mobility_per_district_day = pd.merge(external_mobility, internal_mobility,
                                    on = ['date', 'orig_ags5'])

#rename columns:
mobility_per_district_day.rename(columns = {count_col_name : count_col_name + '_internal',
                                           'orig_ags5' : 'districtId'}, 
                                 inplace=True)

#drop redundant dest_ags5 column:
mobility_per_district_day.drop(columns = 'dest_ags5', inplace=True)

#add total_mobility as sum:
count_cols = [count_col_name + suffix for suffix in ['_outgoing','_incoming','_internal']]
mobility_per_district_day['total_mobility'] = mobility_per_district_day[count_cols].sum(axis=1)

Save mobility per district and day to CSV, again using the identifier:

In [21]:
file_name = identifier + 'mobility_per_district_and_day.csv'
mobility_per_district_day.to_csv(file_name, index=False)